# Bond types - pricing, duration and yield to maturity

In this project, we will look at the different bond types, looking at the key differences in how the price, bond duration and yield to maturity are calculated. 

## Import required libraries

For this project, we need the following libraries:
* `numpy` for working with arrays and mathematical operations
* `scipy.optimize` to provide optimization algorithms

In [1]:
import numpy as np
from scipy.optimize import newton

## Bond class

In [2]:
class bond:
    def __init__(self, face_value, maturity, freq, coupon_rate):
        self.face_value = face_value
        self.maturity = maturity
        self.freq = freq
        self.coupon_rate = coupon_rate
        self.present_value = None
        self.price = None
        self.yield_to_maturity = None
    
    def bond_price(self, yield_rate):
        periods = self.freq * self.maturity
        coupon_payment = self.face_value * self.coupon_rate / self.freq
        coupon_payments = np.repeat(coupon_payment, periods)
        coupon_payments[-1] += self.face_value
        present_value = []
        
        for i in range(periods):
            payment = coupon_payments[i] / ((1 + (yield_rate/self.freq))**(i+1))
            present_value.append(payment)
        
        self.present_value = present_value
        
        self.price = sum(present_value)
        
        return self.price
    
    def duration(self, macaulay=True):
        period = self.maturity / self.freq
        periods = self.maturity * self.freq
        periods_ordered = np.linspace(period, self.maturity, periods)

        duration = 0
        for i in range(periods):
            duration += self.present_value[i] * periods_ordered[i] / self.price
        
        if macaulay == True:
            return duration
        elif macaulay == False:
            if not hasattr(self, 'yield_to_maturity'):
                print("Yield to maturity not calculated")
                return
            else: 
                modified = duration / (1 + (self.yield_to_maturity / self.freq))
                return modified
        else:
            raise TypeError("The modified parameter must take a Boolean value: either True or False")
    
    def yield_to_maturity(self):
        ytm_func = lambda y: self.bond_price(y) - self.face_value
        ytm = newton(ytm_func, 0.05)
        self.yield_to_maturity = ytm
        
        return ytm

## Government and municipal bonds

A government bond is a debt security issued by a government to support government spending and obligations. Government bonds issued by national governments are often considered low-risk investments since the issuing government backs them. However, because of their lower risk, they also carry relatively lower yields. Government bonds assist in funding deficits in the government budgets and are used to raise capital for various projects such as infrastructure spending.

Common examples of these government bonds are: 
* USA - Treasury Bonds
* UK - Gilts
* Germany - Bunds
* France - OATs
* Japan - JGBs
* Italy - BTPs
* Canada - Canada Bonds

Municipal government bonds are extremely similar, however they are issued by local governments to fund projects such as infrastructure, libraries or parks. These bonds often carry certain tax advantages and exemptions for investors. 

In [3]:
class GovernmentBond(bond):
    def __init__(self, face_value, maturity, freq, coupon_rate, credit_rating):
        super().__init__(face_value, maturity, freq, coupon_rate)
        self.credit_rating = credit_rating
        self.default_risk = None
    
    def calculate_default_risk(self):
        if self.credit_rating == "AAA":
            default_risk = 0.1
        elif self.credit_rating == "AA":
            default_risk = 0.25
        elif self.credit_rating == "A":
            default_risk = 0.5
        elif self.credit_rating == "BBB":
            default_risk = 1.0
        elif self.credit_rating == "BB":
            default_risk = 2.0
        elif self.credit_rating == "B":
            default_risk = 5.0
        elif self.credit_rating == "CCC":
            default_risk = 10.0
        elif self.credit_rating == "CC":
            default_risk = 20.0
        elif self.credit_rating == "C":
            default_risk = 50.0
        else:
            print("Enter a valid credit rating")
            
        # Add to default risk attribute
        self.default_risk = default_risk
        
        print(f"The default risk for a bond with a credit rating {self.credit_rating}:{default_risk}%")
        
    def expected_loss(self):
        if not hasattr(self, 'default_risk'):
            print("Default risk not calculated")
            return
        
        expected_loss = self.face_value * self.default_risk
        print(f"The expected loss on this bond is {expected_loss:,.2f}")

## Corporate bonds

Below, we define a `CorporateBond` class that inherits from our government bond class, the model also includes an additional `credit_spread` method. The credit spread is the difference in the yield to maturity of two bonds with the same maturity, but different credit ratings.

In [4]:
class CorporateBond(GovernmentBond):
    def __init__(self, face_value, maturity, freq, coupon_rate, credit_rating, default_risk, treasury_rate):
        super().__init__(face_value, maturity, freq, coupon_rate, credit_rating, default_risk)
        self.treasury_rate = treasury_rate
        
    def credit_spread(self, risk_free_rate):
        if not hasattr(self, 'default_risk'):
            print("Default risk not calculated")
            return
        
        credit_spread = self.yield_to_maturity - self.treasury_rate
        print(f"The credit spread on the bond is {credit_spread:,.2f}")

## Junk bond

Junk bonds are very similar to regular corporate bonds, both represent debt issued by the firm with the promise to pay interest and return the principal at maturity. Junk bonds differ because of the issuers' poorer credit quality. Companies that issue junk bonds are typically start-ups or companies that are struggling financially. 

Junk bonds carry more risk than regular corporate bonds, and therefore investors demand a higher yield on the bonds. Companies are willing to pay this higher yield because they need to attract investors to fund thier operations.

## Zero-coupon bond class

Zero-coupon bonds differ from the average bonds, as they do not pay regular coupon payments but instead trade at a deep discount, rendering a profit at maturity. Because they offer the entire payment at maturity, zero-coupon bonds tend to fluctuate in price much more than coupon bonds.

The interest earned on zero-coupon bond is an imputed interest, meaning it is estimated and not an established rate, it is sometmes referred to as the 'phantom rate'. The difference between the face value and current price represents the interest that compounds automatically until the bond matures.

### Mathematical formulas

**For Price**
$$\text{Price} = \frac{\text{Face Value}}{(1 + \text{YTM})^\text{maturity}}$$

**For Duration**
$$\text{Duration} =  \frac{Maturity}{1 + YTM}$$

In [5]:
class zero_coupon_bond:
    def __init__(self, face_value, maturity, yield_to_maturity=None, market_price=None):
        self.face_value = face_value
        self.maturity = maturity
        self.yield_to_maturity = yield_to_maturity
        self.market_price = market_price
        self.intrinsic_price = None
    
    def price(self):
        if self.face_value is not None and self.yield_to_maturity is not None and self.maturity is not None:
            price = self.face_value / (1 + self.yield_to_maturity) ** self.maturity
            self.intrinsic_price = price
            print(f"The price of the bond is {price:,.2f}")
        else:
            print('Missing input(s) to calculate the zero coupon price.')
            return None
    
    def duration(self, yield_to_maturity):
        # Define the attribute
        self.yield_to_maturity = yield_to_maturity
        
        if self.yield_to_maturity is not None and self.maturity is not None:
            duration = self.maturity / (1 + self.yield_to_maturity)
            print(f"The duration of the bond is {duration:,.2f}")
        else:
            print('Missing input(s) to calculate the zero coupon duration.')
            return None
        
    def ytm(self):
        if self.face_value is not None and self.market_price is not None and self.maturity is not None:
            ytm = (self.face_value / self.market_price) * (1 / self.maturity) - 1
            self.yield_to_maturity = ytm
        else:
            print('Missing input(s) to calculate the zero coupon yield to maturity.')  
            return None

## Convertible bonds

Convertible bonds offer an additional option to convert the face value of the bond into stock at predetermined conversion ratio at any point prior to expiry. Convertible bonds offer the opportunity for increased returns when the stock price increases and the conversion option becomes more valuable. On the other hand, if the bond issuer's stock price decreases, the bondholder may choose to hold onto the bond and receive the fixed interest payments, as the conversion option would less valuable. 

It is worth noting that in exchange for this conversion option, these bonds typically have lower coupon rates than non-convertible bonds from the same issuer, as the conversion option provides additional value to the bondholder.

### Formulas

**For price:**

$$\text{Price} = \text{Bond Value} + \text{Conversion Value}$$

$$\text{Conversion Value} = \text{Conversion Ratio}\cdot\text{Stock Price}$$

*Note: there are alternative ways to calculate the value of convertible bonds, which range in complexity, for simplicity  we will only used the formula mentioned above.*

In [7]:
class ConvertibleBond(bond):
    def __init__(self,  face_value, maturity, freq, coupon_rate, conversion_ratio):
        super().__init__(self,  face_value, maturity, freq, coupon_rate)
        self.conversion_ratio = conversion_ratio
        
    def bond_price(self, yield_rate, stock_price):
        conversion_value = self.conversion_ratio * stock_price
        bond_value = super().bond_price(yield_rate)
        price = max(bond_value, conversion_value)
        self.price = price
        
        return price
    
    def duration(self, macaulay=True):
        raise AttributeError("Duration method is not available")

*Note: In our class, we decided to not make the duration method available as it would require more complex calculations than I currently have the coding ability to implement.*

## Callable bond

A callable bond, also known as a redeemable bond, is a bond that the issuer may redeem before it reaches the stated maturity date. A callable bond allows the issuing company to pay off their debt early. They may choose to do if market interest rates move lower, allowing them to re-borrow at a more beneficial rate. 

Callable bonds compensate investors for that potential outcome, as they offer a more attractive interest rate or coupon rate due to their callable nature.

In [8]:
class CallableBond(bond):
    def __init__(self,  face_value, maturity, freq, coupon_rate):
        super().__init__(face_value, maturity, freq, coupon_rate)
        self.call_premium = None
        self.yield_rate = None
        self.price = None
        
    def bond_price(self, yield_rate, call_premium):
        self.yield_rate = yield_rate
        self.call_premium = call_premium
        bond_value = super().bond_price(yield_rate)
        price = bond_value - call_premium
        self.price = price
        return price
    
    def duration(self):
        if not hasattr(self, 'yield_rate'):
            print("Please run price function and pass yield rate")
            return
        if not hasattr(self, 'call_premium'):
            print("Please run price function and pass call premium")
            return
        
        duration = super().duration(True) - (self.call_premium / self.price) * (1 + self.yield_to_maturity) / self.yield_to_maturity
        return duration